In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from sklearn.model_selection import train_test_split

In [2]:
class DataPreprocessor:
    def __init__(self, filepath):
        self.filepath = filepath
        self.label_encoder = LabelEncoder()

    def load_data(self):
        # Ensure headers are correctly used
        self.data = pd.read_excel(self.filepath, header=0)
        self.features = self.data.iloc[:, :-1]  # Exclude the last column which is 'Condition'
        self.labels = self.data.iloc[:, -1]  # Last column is 'Condition'

    def clean_data(self):
        # Apply to_numeric conversion and coerce errors to handle non-numeric data
        self.features = self.features.apply(pd.to_numeric, errors='coerce')
        # Fill NaN values in features with the mean of each column
        self.features = self.features.fillna(self.features.mean())

    def encode_labels(self):
        # Convert labels to numeric, dropping NaN entries
        self.labels = pd.to_numeric(self.labels, errors='coerce')
        # Drop NaN values from labels and corresponding rows in features
        valid_indices = self.labels.dropna().index
        self.labels = self.labels.loc[valid_indices].astype(int)
        self.features = self.features.loc[valid_indices]
        # Fit and transform labels
        self.labels = self.label_encoder.fit_transform(self.labels)

    def scale_features(self):
        scaler = StandardScaler()
        self.features = scaler.fit_transform(self.features)

    def split_data(self, test_size=0.2, random_state=42):
        return train_test_split(self.features, self.labels, test_size=test_size, random_state=random_state)

    def preprocess(self):
        self.load_data()
        self.clean_data()
        self.encode_labels()
        self.scale_features()
        return self.split_data()


In [3]:
class RNNFaultModel:
    def __init__(self, input_shape, num_outputs):
        self.model = tf.keras.Sequential([
            tf.keras.layers.SimpleRNN(50, return_sequences=True, input_shape=input_shape),
            tf.keras.layers.SimpleRNN(50),
            tf.keras.layers.Dense(num_outputs, activation='softmax')
        ])
        self.model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    def train(self, X_train, y_train, epochs=800):
        self.model.fit(X_train, y_train, epochs=epochs)

    def evaluate(self, X_test, y_test):
        return self.model.evaluate(X_test, y_test)

    def predict(self, X):
        return self.model.predict(X)

In [4]:
class RNNFaultModelManager:
    def __init__(self, input_shape, num_conditions):
        self.models = {}
        self.input_shape = input_shape
        self.num_conditions = num_conditions

    def train_models(self, X_train, y_train, conditions):
        for condition in conditions:
            print(f"Training model for condition: {condition}")
            y_condition = (y_train == condition).astype(int)
            model = RNNFaultModel(input_shape=self.input_shape, num_outputs=self.num_conditions)
            model.train(X_train, y_condition, epochs=800)
            self.models[condition] = model

    def predict_models(self, X_test):
        predictions = {}
        for condition, model in self.models.items():
            predictions[condition] = model.predict(X_test)
        return predictions

In [5]:
# from pgmpy.models import BayesianModel
# from pgmpy.factors.discrete import TabularCPD
# from pgmpy.inference import VariableElimination

# class DynamicBayesianNetwork:
#     def __init__(self, conditions):
#         self.conditions = conditions
#         self.model = BayesianModel()
#         self.setup_model()
#         # Initialize the inference object right after setting up the model
#         self.inference = VariableElimination(self.model)
    
#     def setup_model(self):
#         # Condition nodes are named based on conditions to ensure they are valid node names
#         condition_nodes = ['Condition_' + str(cond) for cond in self.conditions]
#         self.model.add_nodes_from(['State'] + condition_nodes)
#         for condition_node in condition_nodes:
#             self.model.add_edge('State', condition_node)
        
#         # Creating and adding CPDs
#         cpd_state = TabularCPD(variable='State', variable_card=2, values=[[0.5], [0.5]])
#         self.model.add_cpds(cpd_state)
#         for condition_node in condition_nodes:
#             cpd_fault = TabularCPD(variable=condition_node, variable_card=2,
#                                    values=[[0.95, 0.05], [0.05, 0.95]],
#                                    evidence=['State'], evidence_card=[2])
#             self.model.add_cpds(cpd_fault)
        
#         assert self.model.check_model(), "Model has configuration errors"

#     def integrate_with_dbn(self, residuals):
#         evidence_dict = {}
#         for condition, residual_array in residuals.items():
#             # Here we use the mean of the residuals to create a simple evidence threshold
#             mean_residual = np.mean(residual_array)
#             evidence_dict['Condition_' + str(condition)] = int(mean_residual > 0.5)
        
#         updated_beliefs = self.inference.query(variables=list(evidence_dict.keys()), evidence=evidence_dict)
#         return updated_beliefs

#     def get_most_probable_fault(self, evidence):
#         updated_beliefs = self.update_probabilities(evidence)
#         max_prob = 0
#         most_probable_fault = None
#         for condition, belief in updated_beliefs.items():
#             prob_of_fault = belief.values[1]
#             if prob_of_fault > max_prob:
#                 max_prob = prob_of_fault
#                 most_probable_fault = condition
#         return most_probable_fault, max_prob
class DynamicBayesianNetwork:
    def __init__(self, conditions):
        self.conditions = conditions
        self.model = BayesianModel()
        self.setup_model()
        self.inference = VariableElimination(self.model)
    
    def setup_model(self):
        # Assuming 'State' influences all conditions but we only observe conditions
        condition_nodes = ['Condition_' + str(cond) for cond in self.conditions]
        self.model.add_nodes_from(['State'] + condition_nodes)
        for condition_node in condition_nodes:
            self.model.add_edge('State', condition_node)
        
        # CPDs can be more complex based on your actual model requirements
        cpd_state = TabularCPD(variable='State', variable_card=2, values=[[0.7], [0.3]])
        self.model.add_cpds(cpd_state)
        for condition_node in condition_nodes:
            cpd = TabularCPD(variable=condition_node, variable_card=2,
                             values=[[0.95, 0.05], [0.05, 0.95]],
                             evidence=['State'], evidence_card=[2])
            self.model.add_cpds(cpd)

        assert self.model.check_model(), "Model configuration has errors."

    def integrate_with_dbn(self, residuals):
        evidence_dict = {}
        for condition, residual_array in residuals.items():
            # Simple thresholding example: if mean residual > 0.5, condition is likely faulty (1)
            evidence_dict['Condition_' + str(condition)] = int(np.mean(residual_array) > 0.5)

        # We query for the 'State' to understand the overall system state
        return self.inference.query(variables=['State'], evidence=evidence_dict)


In [6]:
import pandas as pd
import numpy as np

class ModelEvaluator:
    def __init__(self, models, X_test, y_test):
        self.models = models
        self.X_test = X_test
        self.y_test = y_test

    def evaluate_models(self):
        evaluations = {}
        for condition, model in self.models.items():
            loss, accuracy = model.evaluate(self.X_test, self.y_test == condition)
            evaluations[condition] = {'loss': loss, 'accuracy': accuracy}
        return evaluations

    def predict_random_samples(self, n_samples):
        idx = np.random.choice(self.X_test.shape[0], n_samples, replace=False)
        all_predictions = []
        
        # Collect predictions from all models
        for condition, model in self.models.items():
            predictions = model.predict(self.X_test[idx])
            predicted_classes = np.argmax(predictions, axis=1)
            all_predictions.append(predictions[:, 1])  # Assuming binary output for each model
        
        # Determine the most likely condition for each sample
        all_predictions = np.array(all_predictions).T
        predicted_condition_indices = np.argmax(all_predictions, axis=1)
        predicted_conditions = [list(self.models.keys())[index] for index in predicted_condition_indices]
        actual_conditions = self.y_test[idx]

        df = pd.DataFrame({
            'Sample Index': idx,
            'Predicted Condition': predicted_conditions,
            'Actual Condition': actual_conditions
        })
        return df

    def display_sample_predictions(self, n_samples):
        df = self.predict_random_samples(n_samples)
        print(df)

    def accuracy_of_samples(self, n_samples):
        df = self.predict_random_samples(n_samples)
        correct_predictions = df['Predicted Condition'] == df['Actual Condition']
        accuracy = correct_predictions.mean()
        print(f"Accuracy of the sample predictions: {accuracy:.2%}")


In [7]:
class ResidualCalculator:
    def __init__(self, models, X_test, y_test):
        self.models = models
        self.X_test = X_test
        self.y_test = y_test

    def calculate_residuals(self):
        residuals = {}
        for condition, model in self.models.items():
            predictions = model.predict(self.X_test)
            actual = (self.y_test == condition).astype(int)
            residuals[condition] = np.abs(predictions - actual[:, np.newaxis])  # assuming softmax output
        return residuals


In [8]:
def main():
    filepath = r'C:\Users\SIDDHARTH SINGH\Downloads\Untitled Folder\Merged_dataset.xlsx'
    data_preprocessor = DataPreprocessor(filepath)
    X_train, X_test, y_train, y_test = data_preprocessor.preprocess()
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    conditions = np.unique(y_train)
    conditions_str = [str(cond) for cond in conditions]

    model_manager = RNNFaultModelManager(input_shape=(X_train.shape[1], 1), num_conditions=len(conditions))
    model_manager.train_models(X_train, y_train, conditions)

    evaluator = ModelEvaluator(model_manager.models, X_test, y_test)
    initial_evaluations = evaluator.evaluate_models()
    print("Initial Model Evaluations:", initial_evaluations)

    residual_calculator = ResidualCalculator(model_manager.models, X_test, y_test)
    residuals = residual_calculator.calculate_residuals()

    dbn = DynamicBayesianNetwork(conditions_str)
    state_beliefs = dbn.integrate_with_dbn(residuals)
    state_prob = state_beliefs.values

    for condition in conditions:
        if state_prob[1] > 0.5:
            print(f"Re-training model for condition {condition} due to high probability of system fault.")
            y_condition = (y_train == condition).astype(int)
            model_manager.models[str(condition)].train(X_train, y_condition, epochs=800)

    final_evaluations = evaluator.evaluate_models()
    print("Final Model Evaluations after DBN Feedback:", final_evaluations)

    evaluator.display_sample_predictions(100)
    evaluator.accuracy_of_samples(100)  # Example: Calculate accuracy for 100 randomly selected test samples

if __name__ == "__main__":
    main()


Training model for condition: 0
Epoch 1/1000
201/201 [==============================] - 2s 4ms/step - loss: 0.2858 - accuracy: 0.9311
Epoch 2/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0444 - accuracy: 0.9811
Epoch 3/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0914 - accuracy: 0.9636
Epoch 4/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0275 - accuracy: 0.9898
Epoch 5/1000
201/201 [==============================] - 1s 5ms/step - loss: 0.0155 - accuracy: 0.9948
Epoch 6/1000
201/201 [==============================] - 1s 7ms/step - loss: 0.0075 - accuracy: 0.9977
Epoch 7/1000
201/201 [==============================] - 1s 6ms/step - loss: 0.0044 - accuracy: 0.9983
Epoch 8/1000
201/201 [==============================] - 1s 5ms/step - loss: 9.6025e-04 - accuracy: 1.0000
Epoch 9/1000
201/201 [==============================] - 1s 5ms/step - loss: 7.0772e-04 - accuracy: 1.0000
Epoch 10/1000
201/201 [===================

201/201 [==============================] - 1s 7ms/step - loss: 2.0992e-07 - accuracy: 1.0000
Epoch 78/1000
201/201 [==============================] - 1s 5ms/step - loss: 1.9218e-07 - accuracy: 1.0000
Epoch 79/1000
201/201 [==============================] - 1s 4ms/step - loss: 1.1811e-07 - accuracy: 1.0000
Epoch 80/1000
201/201 [==============================] - 1s 5ms/step - loss: 8.7512e-08 - accuracy: 1.0000
Epoch 81/1000
201/201 [==============================] - 1s 5ms/step - loss: 7.5630e-08 - accuracy: 1.0000
Epoch 82/1000
201/201 [==============================] - 1s 5ms/step - loss: 6.7957e-08 - accuracy: 1.0000
Epoch 83/1000
201/201 [==============================] - 1s 5ms/step - loss: 6.2985e-08 - accuracy: 1.0000
Epoch 84/1000
201/201 [==============================] - 1s 5ms/step - loss: 5.7696e-08 - accuracy: 1.0000
Epoch 85/1000
201/201 [==============================] - 1s 5ms/step - loss: 5.1885e-08 - accuracy: 1.0000
Epoch 86/1000
201/201 [============================

201/201 [==============================] - 1s 4ms/step - loss: 0.0103 - accuracy: 0.9969
Epoch 154/1000
201/201 [==============================] - 1s 5ms/step - loss: 5.0675e-04 - accuracy: 1.0000
Epoch 155/1000
201/201 [==============================] - 1s 7ms/step - loss: 0.0468 - accuracy: 0.9847
Epoch 156/1000
201/201 [==============================] - 1s 6ms/step - loss: 0.0582 - accuracy: 0.9717
Epoch 157/1000
201/201 [==============================] - 1s 6ms/step - loss: 0.0068 - accuracy: 0.9977
Epoch 158/1000
201/201 [==============================] - 1s 5ms/step - loss: 0.0062 - accuracy: 0.9984
Epoch 159/1000
201/201 [==============================] - 1s 6ms/step - loss: 0.0030 - accuracy: 0.9994
Epoch 160/1000
201/201 [==============================] - 1s 7ms/step - loss: 4.7259e-04 - accuracy: 1.0000
Epoch 161/1000
201/201 [==============================] - 2s 10ms/step - loss: 1.3665e-04 - accuracy: 1.0000
Epoch 162/1000
201/201 [==============================] - 1s 7ms/s

201/201 [==============================] - 1s 5ms/step - loss: 2.4844e-08 - accuracy: 1.0000
Epoch 230/1000
201/201 [==============================] - 1s 4ms/step - loss: 2.3838e-08 - accuracy: 1.0000
Epoch 231/1000
201/201 [==============================] - 1s 4ms/step - loss: 2.2739e-08 - accuracy: 1.0000
Epoch 232/1000
201/201 [==============================] - 1s 5ms/step - loss: 2.1007e-08 - accuracy: 1.0000
Epoch 233/1000
201/201 [==============================] - 1s 4ms/step - loss: 1.9294e-08 - accuracy: 1.0000
Epoch 234/1000
201/201 [==============================] - 1s 4ms/step - loss: 1.5383e-08 - accuracy: 1.0000
Epoch 235/1000
201/201 [==============================] - 1s 5ms/step - loss: 1.3893e-08 - accuracy: 1.0000
Epoch 236/1000
201/201 [==============================] - 1s 4ms/step - loss: 1.2608e-08 - accuracy: 1.0000
Epoch 237/1000
201/201 [==============================] - 1s 4ms/step - loss: 1.1882e-08 - accuracy: 1.0000
Epoch 238/1000
201/201 [===================

201/201 [==============================] - 1s 5ms/step - loss: 1.4013e-05 - accuracy: 1.0000
Epoch 306/1000
201/201 [==============================] - 1s 5ms/step - loss: 1.1373e-05 - accuracy: 1.0000
Epoch 307/1000
201/201 [==============================] - 1s 5ms/step - loss: 9.5676e-06 - accuracy: 1.0000
Epoch 308/1000
201/201 [==============================] - 1s 5ms/step - loss: 8.2785e-06 - accuracy: 1.0000
Epoch 309/1000
201/201 [==============================] - 1s 4ms/step - loss: 7.2650e-06 - accuracy: 1.0000
Epoch 310/1000
201/201 [==============================] - 1s 5ms/step - loss: 6.4087e-06 - accuracy: 1.0000
Epoch 311/1000
201/201 [==============================] - 1s 4ms/step - loss: 5.6690e-06 - accuracy: 1.0000
Epoch 312/1000
201/201 [==============================] - 1s 5ms/step - loss: 5.0090e-06 - accuracy: 1.0000
Epoch 313/1000
201/201 [==============================] - 1s 5ms/step - loss: 4.4224e-06 - accuracy: 1.0000
Epoch 314/1000
201/201 [===================

...

Epoch 772/1000
201/201 [==============================] - 1s 5ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 773/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 774/1000
201/201 [==============================] - 1s 5ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 775/1000
201/201 [==============================] - 1s 7ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 776/1000
201/201 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 777/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 778/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 779/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 780/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 781/1000
201/201 [====

201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 848/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 849/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 850/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 851/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 852/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 853/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 854/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 855/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 856/1000
201/201 [===================

Epoch 923/1000
201/201 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 924/1000
201/201 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 925/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 926/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 927/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 928/1000
201/201 [==============================] - 1s 5ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 929/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 930/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 931/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 932/1000
201/201 [====

201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 999/1000
201/201 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 1000/1000
51/51 [==============================] - 0s 2ms/step - loss: 3.2486e-07 - accuracy: 1.0000
Initial Model Evaluations: {0: {'loss': 1.5082312074810034e-06, 'accuracy': 1.0}, 1: {'loss': 1.4891851640719267e-10, 'accuracy': 1.0}, 2: {'loss': 0.0, 'accuracy': 1.0}, 3: {'loss': 0.02733875811100006, 'accuracy': 0.9893816113471985}, 4: {'loss': 0.0, 'accuracy': 1.0}, 5: {'loss': 0.03128620609641075, 'accuracy': 0.9887570142745972}, 6: {'loss': 0.013084197416901588, 'accuracy': 0.9993754029273987}, 7: {'loss': 0.04325157031416893, 'accuracy': 0.9875078201293945}, 8: {'loss': 0.04537678509950638, 'accuracy': 0.9818863272666931}, 9: {'loss': 0.022811684757471085, 'accuracy': 0.993129312992096}, 10: {'loss': 0.0, 'accuracy': 1.0}, 11: {'loss': 0.02295404113829136, 'accuracy': 0.99875

51/51 [==============================] - 0s 2ms/step - loss: 3.2486e-07 - accuracy: 1.0000
Final Model Evaluations after DBN Feedback: {0: {'loss': 1.5082312074810034e-06, 'accuracy': 1.0}, 1: {'loss': 1.4891851640719267e-10, 'accuracy': 1.0}, 2: {'loss': 0.0, 'accuracy': 1.0}, 3: {'loss': 0.02733875811100006, 'accuracy': 0.9893816113471985}, 4: {'loss': 0.0, 'accuracy': 1.0}, 5: {'loss': 0.03128620609641075, 'accuracy': 0.9887570142745972}, 6: {'loss': 0.013084197416901588, 'accuracy': 0.9993754029273987}, 7: {'loss': 0.04325157031416893, 'accuracy': 0.9875078201293945}, 8: {'loss': 0.04537678509950638, 'accuracy': 0.9818863272666931}, 9: {'loss': 0.022811684757471085, 'accuracy': 0.993129312992096}, 10: {'loss': 0.0, 'accuracy': 1.0}, 11: {'loss': 0.02295404113829136, 'accuracy': 0.9987508058547974}, 12: {'loss': 0.0, 'accuracy': 1.0}, 13: {'loss': 0.0, 'accuracy': 1.0}, 14: {'loss': 0.012028633616864681, 'accuracy': 0.9968769550323486}, 15: {'loss': 3.248557618462655e-07, 'accuracy'